In [28]:
# T1-23. Drop Duplicates / #중복제거 #결측치 #10번째값으로채움
# f1의 결측치를 채운 후 age 컬럼의 중복 제거 전과 후의 'f1' 중앙값 차이를 구하시오
# - 결측치는 f1의 데이터 중 내림차순 정렬 후 10번째 값으로 채움
# - 중복 데이터 발생시 뒤에 나오는 데이터를 삭제함
# - 최종 결과값은 절대값으로 출력
# 풀이

import pandas as pd

df = pd.read_csv('./data/basic1.csv')
# print(df.head(3))
# print(df.isnull().sum()) # f1(31), f3(95)
# print(df.shape) # (100, 8)

df = df.sort_values('f1', ascending=False)
# print(df.head(3))
# print(df.iloc[:10])
# print(df.iloc[9]['f1'])

df['f1'] = df['f1'].fillna(df.iloc[9]['f1'])
# print(df.isnull().sum())

m1 = df['f1'].median()

# df['age'].fillna()
# print(dir(df['age']))
# print(help(df['age'].drop_duplicates))
# print(df['age'].shape)
idx = df['age'].drop_duplicates(keep='first').index
m2 = df.iloc[idx]['f1'].median()

answer = abs(m1 - m2)
print(answer)

0.5


In [29]:
# 해답

import pandas as pd

df = pd.read_csv('./data/basic1.csv')

top10 = df['f1'].sort_values(ascending=False).iloc[9]
# print(top10)
df['f1'] = df['f1'].fillna(top10)

result1 = df['f1'].median()

df = df.drop_duplicates(subset=['age'])

result2 = df['f1'].median()

print(abs(result1 - result2))

0.5


### ==============================

In [38]:
# T1-24. Time-Series5 (Lagged Feature) / #시차데이터 #조건
# 주어진 데이터(basic2.csv)에서 "pv"컬럼으로 1일 시차(lag)가 있는 
# 새로운 컬럼을 만들고(예: 1월 2일에는 1월 1일 pv데이터를 넣고, 
# 1월 3일에는 1월 2일 pv데이터를 넣음),새로운 컬럼의 1월 1일은 다음날(1월 2일)
# 데이터로 결측치를 채운 다음, Events가 1이면서 Sales가 1000000이하인 
# 조건에 맞는 새로운 컬럼 합을 구하시오
# 풀이

import pandas as pd

df = pd.read_csv('./data/basic2.csv')
print(df.head(3))
# print(df.isnull().sum()) # UV(46)
# print(df.shape) # (730, 5)

df['PV1'] = df['PV'][:len(df)-1]
# print(df.tail())
df['PV1'][1:] = df['PV1'][:len(df)-1]
# print(df.head())

df = df[(df['Events'] == 1) & (df['Sales'] <= 1000000)]
answer = df['PV1'].sum()
print(answer)

         Date     Sales      PV      UV  Events
0  2022-01-01  22711525  397349  4421.0       1
1  2022-01-02     36779    3969  2289.0       0
2  2022-01-03  13943875  373890     NaN       1
1894876.0


C:\Users\user\AppData\Local\Temp\ipykernel_1848\3380501961.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['PV1'][1:] = df['PV1'][:len(df)-1]


In [39]:
# 해답

import pandas as pd

df = pd.read_csv('./data/basic2.csv')

df['previous_PV'] = df['PV'].shift(1)

df['previous_PV'] = df['previous_PV'].fillna(method = 'bfill')
# df.head()

cond = (df['Events'] == 1) & (df['Sales'] <= 1000000)
print(df[cond]['previous_PV'].sum())

1894876.0


### ==============================

In [50]:
# T1-25. 성격유형 E형 찾기 / #문자 슬라이싱
# basic1 데이터에서 f4가 E로 시작하면서 부산에 살고 20대인 사람은 몇 명일까요?
# 풀이

import pandas as pd

df = pd.read_csv('./data/basic1.csv')
# print(df.head(3))
# print(df.isnull().sum()) # f1(31), f3(95)
# print(df.shape) # (100, 8)

# print(dir(df['f4']))
ls = []
for i in list(df['f4']):
    ls.append(i[:1])
# print(ls)

df['f4_1'] = ls
# print(df.head())

cond1 = df['f4_1'] == 'E'
cond2 = df['city'] == '부산'
cond3 = df['age'] >= 20
cond4 = df['age'] < 30

# print(df[cond1 & cond2 & cond3 & cond4])
# print(df[cond1 & cond2])

answer = len(df[cond1 & cond2 & cond3 & cond4])
print(answer)

0


In [51]:
# 해답

import pandas as pd

df = pd.read_csv('./data/basic1.csv')

df['EI'] = df['f4'].str[:1]
cond1 = df['EI'] == "E"
cond2 = df['city'] == "부산"
cond3 = (df['age'] >= 20) & (df['age'] < 30)

print(len(df[cond1 & cond2 & cond3]))

0


### ==============================

In [68]:
# T2-1. Titanic (Classification) / 타이타닉
# 학습용 데이터 (X_train, y_train)을 이용하여 생존 예측 모형을 만든 후, 
# 이를 평가용 데이터(X_test)에 적용하여 얻은 예측값을 다음과 같은 형식의 
# CSV파일로 생성하시오(제출한 모델의 성능은 accuracy 평가지표에 따라 채점)
# 풀이

import pandas as pd

train = pd.read_csv('./data/titanic_train.csv')
test = pd.read_csv('./data/titanic_test.csv')

# print(train.head(3))
# print(train.isnull().sum()) # Age(177), Cabin(687), Embarked(2)
# print(train.shape) # (891, 12)

# print(test.head(3))
print(test.isnull().sum())
# print(test.shape) # (418, 11), survived 없음

# EDA (train, test)


# 결측치 처리
train['Age'] = train['Age'].fillna(train['Age'].mean())
train = train.drop(columns='Cabin')
# print(train['Embarked'].value_counts())
# print(train['Embarked'].mode())
train['Embarked'] = train['Embarked'].fillna(train['Embarked'].mode()[0])
# print(help(train['Age'].mode))
print(train.isnull().sum())

# 스케일링
# 모델 만들기
# 모델 fit
# predict
# 제출

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64
PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       0
dtype: int64
